## **Load libraries and data**

In [1]:
!pip install --upgrade pythainlp
!pip install pyLDAvis
!pip install emoji

In [2]:
import pandas as pd
import string
import emoji
import re
import pythainlp
from nltk.tokenize import RegexpTokenizer
from pythainlp.corpus.common import thai_words
from pythainlp import Tokenizer, word_tokenize
from pythainlp.util import normalize
from pythainlp import correct
import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [3]:
pythainlp.__version__

'2.3.1'

In [4]:
df = pd.read_csv('CustomerReviews.csv')

In [5]:
df.tail()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating
16,17,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,ployynp,บุฟเฟ่ต์ชาบูและพิซซ่าไม่อั้นในราคา 199 บาท เน้...,หลังจากที่เคยลองสาขายูเนี่ยนมอลล์ไป รอบนี้มาที...,4.0
17,18,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,27a91236fe5e4559a4f097c97a480781,ร้านบุฟเฟ่ต์ ราคามิตรภาพ อยู่ชั้น4 ติดโรงหนัง ...,ร้านบุฟเฟ่ต์ที่มีโปรโมชั่นหัวละ199บาท ไม่รวมน้...,4.0
18,19,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,0b81d251e4db486f9bcdba73b374ed99,ของหลากหลาย ปนๆ งงๆ นิดหน่อย,เคยรู้จักร้านนี้จากที่ union mall ไม่เคยได้ลอง...,3.0
19,20,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,40e0e087f3914fd49a8933b5a29936ca,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,5.0
20,21,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,41841cb99ea243a3a8d4b006e946c586,แม้จะแปลกบ้าง แต่ก็ถือว่าอยู่ในเกณฑ์ที่ดี มีอา...,ก็ตามที่เขียนเลยครับ ว่า ถ้าจะจ่ายในราคา 199 บ...,NaN


## **Tokenize Words**

In [21]:
stopwords = list(pythainlp.corpus.thai_stopwords())
punctuations = list(string.punctuation)
removed_words = [' ', '  ', '\n', 'ร้าน', '(', ')' , '           ', '!!!!', '++', '..', './', '::', '^^','xx','\u200b',':','-']


screening_words = stopwords + removed_words +punctuations

def tokenize_with_space(sentence):
  sentence = re.sub(r'\d+', '', sentence)
  merged = ''
  words = word_tokenize(normalize(str(sentence)), engine="multi_cut", keep_whitespace=False)
  for word in words:
    #print(word)
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [22]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [23]:
df.tail()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,Review_tokenized,topics,score
16,17,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,ployynp,บุฟเฟ่ต์ชาบูและพิซซ่าไม่อั้นในราคา 199 บาท เน้...,หลังจากที่เคยลองสาขายูเนี่ยนมอลล์ไป รอบนี้มาที...,4.0,"หลังจากที่,เคยลอง,สาขายู,นม,อลล์,รอบนี้,สาขา,เ...",3,0.997854
17,18,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,27a91236fe5e4559a4f097c97a480781,ร้านบุฟเฟ่ต์ ราคามิตรภาพ อยู่ชั้น4 ติดโรงหนัง ...,ร้านบุฟเฟ่ต์ที่มีโปรโมชั่นหัวละ199บาท ไม่รวมน้...,4.0,"ร้านบุฟเฟ่ต์,โปรโมชั่น,หัว,บาท,น้ำ,VAT,ทานวัน,...",3,0.997525
18,19,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,0b81d251e4db486f9bcdba73b374ed99,ของหลากหลาย ปนๆ งงๆ นิดหน่อย,เคยรู้จักร้านนี้จากที่ union mall ไม่เคยได้ลอง...,3.0,"รู้จักร้าน,union,mall,ลอง,กิน,จำได้,ขึ้นใจ,จาก...",0,0.995900
19,20,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,40e0e087f3914fd49a8933b5a29936ca,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,5.0,"อร่อย,มากค่ะ,คุ้มค่า,สมราคา,บุ,ฟเฟ่,หมู,ผัก,น้...",3,0.996967
20,21,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,41841cb99ea243a3a8d4b006e946c586,แม้จะแปลกบ้าง แต่ก็ถือว่าอยู่ในเกณฑ์ที่ดี มีอา...,ก็ตามที่เขียนเลยครับ ว่า ถ้าจะจ่ายในราคา 199 บ...,NaN,"ก็ตามที่,จ่าย,ราคา,บาท,จ่าย,เงินสดนะ,ราคานี้,น...",5,0.994231


In [11]:
from sklearn.feature_extraction.text import CountVectorizer

cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(df['Review_tokenized'])

In [12]:
cvec.vocabulary_

{'CTW': 0,
 'G': 1,
 'HALL': 2,
 'MCC': 3,
 'Menu': 4,
 'Narai': 5,
 'Net': 6,
 'Number': 7,
 'Shabushi': 8,
 'Terminal': 9,
 'VAT': 10,
 'mall': 11,
 'net': 12,
 'pizza': 13,
 'pizzeria': 14,
 'union': 15,
 'ก': 16,
 'กก': 17,
 'กรรมวิธี': 18,
 'กรอบดี': 19,
 'กลยุทธ์': 20,
 'กลับมา': 21,
 'กลายเป็นมา': 22,
 'กั้น': 23,
 'กา': 24,
 'การงดรับ': 25,
 'การบริการก็': 26,
 'การบริการก็ดี': 27,
 'การบริการรสชาติ': 28,
 'การรอ': 29,
 'กาแฟ': 30,
 'กำ': 31,
 'กิน': 32,
 'กี่': 33,
 'กุ้ง': 34,
 'กๆ': 35,
 'ก็ตามที่': 36,
 'ก์': 37,
 'ขนมอร่อย': 38,
 'ขนาด': 39,
 'ขวัญใจ': 40,
 'ของการกิน': 41,
 'ของกิน': 42,
 'ของคาว': 43,
 'ของบน': 44,
 'ของมา': 45,
 'ของสดก็': 46,
 'ของหวาน': 47,
 'ของหวานที่': 48,
 'ของหวานราคา': 49,
 'ของหวานอร่อย': 50,
 'ของอาหาร': 51,
 'ขอมารีวิวร้าน': 52,
 'ขาดสารอาหาร': 53,
 'ขาบูชิต': 54,
 'ขึ้นราคา': 55,
 'ขึ้นใจ': 56,
 'ข่าว': 57,
 'ข้น': 58,
 'ข้อดี': 59,
 'ข้างนอก': 60,
 'ข้าว': 61,
 'ข้าวทานกับ': 62,
 'ข้าวปั้น': 63,
 'ข้าวมันไก่': 64,
 'คน': 65,
 'คนละ': 66,
 '

## **Create Dictionary**

In [13]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [14]:
print(dictionary.token2id.keys())

dict_keys(['การบริการรสชาติ', 'กิน', 'ของอาหาร', 'ข้างนอก', 'ความต้องการ', 'คอ', 'คุณภาพ', 'คุณภาพดี', 'คุ้ม', 'จำกัด', 'ซุป', 'ดำ', 'ดีมาก', 'ถาด', 'ถามราคา', 'น่ารัก', 'น้ำ', 'น้ำจิ้ม', 'บริการ', 'บาท', 'ประทับใจ', 'พนักงาน', 'มาก่อน', 'รสชาติ', 'รสชาติดีนะ', 'สรุป', 'สะอาดสะอ้าน', 'สัน', 'หลากหลาย', 'อร่อย', 'อาหาร', 'อาหารอันนี้', 'เคยกิน', 'เคลม', 'เซ็ท', 'เต้าเจี้ยว', 'เนื้อ', 'เป็ด', 'เผ็ด', 'แต่เพียงพอกับ', 'แบบนี้', 'แปลกใหม่', 'แฟน', 'โดยรวม', 'โมจิ', 'ใส่ใจ', 'ไม่ต้องออก', 'ไอติมก็', 'CTW', 'Terminal', 'การงดรับ', 'การบริการก็', 'คิดถึง', 'ชาบู', 'ญี่ปุ่น', 'ฐาน', 'ดี', 'ทะยอย', 'ทำงานมาก', 'ที่ทาง', 'นาที', 'นึกถึง', 'พนักงานบริการดี', 'พระราม', 'พรีเมียม', 'พาราไดซ์', 'มาตรา', 'มีโอกาสมาทานก่อนร้าน', 'รายละเอียด', 'ร้านก็', 'ลูกค้า', 'สาขา', 'สาขานี้', 'สู้', 'ออร์เดอร์', 'เครื่องดื่ม', 'เค้า', 'เจอ', 'เป็นกันเอง', 'เหวี่ยง', 'แจ้ง', 'โม', 'โม่', 'โอเค', 'กั้น', 'ของหวาน', 'คนละ', 'คาว', 'คุ', 'คุ้มค่า', 'จาน', 'จิ้ม', 'จืด', 'ฉาก', 'ชอบ', 'ชาเขียว', 'ชิ้น', 'ซอย', 'ซอส', 

In [15]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

## **Topic Modeling**

In [16]:
num_topics = 7
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 303 ms, sys: 982 µs, total: 304 ms
Wall time: 318 ms


In [17]:
gensimvis.prepare(model, gensim_corpus, dictionary)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.064902  0.044907       1        1  27.527574
2     -0.075520  0.035480       2        1  24.354738
4      0.009052 -0.051573       3        1  18.499185
0     -0.005764 -0.036460       4        1  13.894164
5      0.006665  0.004683       5        1   8.428995
6      0.003386 -0.000249       6        1   7.267085
1     -0.002720  0.003211       7        1   0.028258, topic_info=      Term       Freq      Total Category  logprob  loglift
16     น้ำ  23.000000  23.000000  Default  30.0000  30.0000
29   อร่อย  17.000000  17.000000  Default  29.0000  29.0000
53    ชาบู  17.000000  17.000000  Default  28.0000  28.0000
147  เลือก  17.000000  17.000000  Default  27.0000  27.0000
105    ทาน  16.000000  16.000000  Default  26.0000  26.0000
..     ...        ...        ...      ...      ...      ...
296     เท   0.001760   6.532855   Topic7  -5.3263  -0.0477
284   ปุระ   0.001759   6.531396   Topic7  -5.3267  -0.0479
286      ม   0.001759   6.531410   Topic7  -5.3267  -0.0479
56      ดี   0.002157  10.420767   Topic7  -5.1230  -0.3114
94     ชอบ   0.001728   8.035147   Topic7  -5.3448  -0.2733

[472 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
331       1  0.910733      net
332       1  0.259534   กลับมา
332       2  0.519068   กลับมา
473       1  0.870195    การรอ
1         1  0.232853      กิน
...     ...       ...      ...
166       1  0.321282    ไอติม
166       2  0.160641    ไอติม
166       3  0.321282    ไอติม
166       4  0.160641    ไอติม
275       2  0.935983  ไอศครีม

[380 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 5, 1, 6, 7, 2])

In [18]:
model.show_topic(3)

[('น้ำ', 0.018658405),
 ('บาท', 0.017083827),
 ('ซุป', 0.015956163),
 ('เลือก', 0.015162739),
 ('เนื้อ', 0.014814438),
 ('ทาน', 0.01473948),
 ('อร่อย', 0.013641984),
 ('ดี', 0.012326624),
 ('กิน', 0.011121657),
 ('ชาบู', 0.010530153)]

In [19]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [20]:
df[['Restaurant', 'Review', 'topics']]

,Restaurant,Review,topics
0,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,2
1,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,0
2,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4
3,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,3
4,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,4
5,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านบุฟเฟ่ ชาบูแนวญี่ปุ่น สายเนื้อหมู เนื้อวัว...,3
6,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,Number 20 : โมโม – พาราไดส์ (สาขาเดอะมอลบางกะป...,6
7,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านชาบูเฟรนไชส์รสชาติดีมากคุ้มค่าเหมาะสมกับรา...,2
8,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,มา านที่ขาบูชิต้องมาตอนหิว ไม่งั้นจะไม่คุ้มนะค...,6
9,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,ใครชอบกุ้งทอดเทมปุระ แค่กุ้งเทมปุระก็คุ้มแล้ว ...,4
